# Cape Basin Super Resolution

A simple notebook to test assorted SR models operating on the Cape Basin dataset

## Preparation

In [1]:
%matplotlib ipympl
import torch
import numpy as np
import hydra, os
from fmod.base.util.config import fmconfig, cfg, ConfigContext
from fmod.model.sres.manager import ResultsAccumulator
from fmod.base.io.loader import TSet
from fmod.base.util.ops import fmbdir, fmtp
from typing import Any, Dict, List, Tuple, Type, Optional, Union
import matplotlib.pyplot as plt

In [2]:
hydra.initialize(version_base=None, config_path="../config")
device = ConfigContext.set_device()


task="sres"
models = [ 'dbpn' ]
dataset="LLC4320-v1"
scenario="s4"
fmconfig( task, models[0], dataset, scenario )
fsize, nmodels = 4.0, len(models)
yscale = "log"

### Results data
Read results summary from previous runs:

In [ ]:
accum_results = ResultsAccumulator(task,dataset,scenario)
plot_data: Dict[str,Tuple[np.ndarray,np.ndarray]] = accum_results.get_plot_data(fmbdir('processed'), models)

### Plot Losses

In [ ]:
with plt.ioff():
	fig, axs = plt.subplots(nrows=nmodels, ncols=1, figsize=[fsize*2,fsize*nmodels], layout="tight")

fig.suptitle('SRes Loss Over Training Epochs', fontsize=14, va="top", y=1.0)
fmt = { TSet.Validation: "b", TSet.Test: "g" }
for ax, model in zip(axs,models):
    x,y = plot_data[model]
    min_loss = {}
    for tset in [ TSet.Validation, TSet.Test ]:
        xp, yp = x[ tset ], y[ tset ]
        min_loss[tset] = yp.min() if (yp.size > 0) else 0.0
        ax.plot( xp, yp, fmt[tset], label=tset.name )
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title(f"Model '{model}':  Validation Loss = {min_loss[TSet.Validation]:.4f}, Test Loss = {min_loss[TSet.Test]:.4f} ")
    ax.legend()
    
plt.show()